In [1]:
!pip install -q transformers==v4.37.1 sentencepiece datasets accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00


In [2]:
import torch
from torch import nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm
from datasets import load_dataset, load_metric
from transformers import MT5ForConditionalGeneration, AutoTokenizer, DataCollatorForSeq2Seq

In [3]:
dataset = load_dataset("el2e10/aya-paraphrase-gujarati")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1001 [00:00<?, ? examples/s]

In [4]:
dataset['train']

Dataset({
    features: ['inputs', 'targets', 'template_lang', 'template_id'],
    num_rows: 1001
})

In [5]:
dataset['train']['inputs'][0]

'નીચેના વાક્યને બીજા સ્વરૂપમાં ફરીથી લખો: "ઘટનાની જાણ થતાં પોલીસ ઘટના સ્થળે પહોંચી હતી અને ઘાયલોને સ્થાનિક હોસ્પિટલમાં દાખલ કરવામાં આવ્યા હતા."'

In [6]:
dataset['train']['targets'][0]

'"પોલીસ ત્યારે ઘટના સ્થળે પહોંચી હતી જ્યારે ઘટનાની જાણ થઇ હતી અને પછી ઘાયલોને સ્થાનિક હોસ્પિટલમાં દાખલ કરવામાં આવ્યા હતા."'

In [7]:
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:515: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [8]:
special_tokens = {"bos_token":"<s>"}
tokenizer.add_special_tokens(special_tokens)
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>', '<pad>']

# Model output test

In [11]:
input = """નીચેના વાક્યને બીજા સ્વરૂપમાં ફરીથી લખો: "ઘટનાની જાણ થતાં પોલીસ ઘટના સ્થળે પહોંચી હતી અને ઘાયલોને સ્થાનિક હોસ્પિટલમાં દાખલ કરવામાં આવ્યા હતા."""
output = """પોલીસ ત્યારે ઘટના સ્થળે પહોંચી હતી જ્યારે ઘટનાની જાણ થઇ હતી અને પછી ઘાયલોને સ્થાનિક હોસ્પિટલમાં દાખલ કરવામાં આવ્યા હતા."""
inputs = tokenizer(input, text_target=output, return_tensors="pt")

tokenizer.batch_decode(model.generate(inputs.input_ids,max_length=50),skip_special_tokens=True)[0]

'<extra_id_0> કરો'

# Dataset splits

In [12]:
datasets_train_test = dataset["train"].train_test_split(test_size=400)
datasets_train_validation = datasets_train_test["test"].shuffle().train_test_split(test_size=200)

In [13]:
datasets_train_test

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets', 'template_lang', 'template_id'],
        num_rows: 601
    })
    test: Dataset({
        features: ['inputs', 'targets', 'template_lang', 'template_id'],
        num_rows: 400
    })
})

In [14]:
datasets_train_validation

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets', 'template_lang', 'template_id'],
        num_rows: 200
    })
    test: Dataset({
        features: ['inputs', 'targets', 'template_lang', 'template_id'],
        num_rows: 200
    })
})

In [15]:
dataset["train"] = datasets_train_test["train"]
dataset["validation"] = datasets_train_validation["train"]
dataset["test"] = datasets_train_validation["test"]

In [16]:
dataset["train"], dataset["validation"], dataset["test"]

(Dataset({
     features: ['inputs', 'targets', 'template_lang', 'template_id'],
     num_rows: 601
 }),
 Dataset({
     features: ['inputs', 'targets', 'template_lang', 'template_id'],
     num_rows: 200
 }),
 Dataset({
     features: ['inputs', 'targets', 'template_lang', 'template_id'],
     num_rows: 200
 }))

In [17]:
max_input_length = 512
max_target_length = 512

def preprocess_data(data):

  inputs = data["inputs"]
  targets = ["<s>" + text for text in data["targets"]]
  model_inputs = tokenizer(inputs,
                           padding=True,
                           max_length=max_input_length,
                           truncation=True)

  # Setup the tokenizer for targets
  labels = tokenizer(targets,
                     padding=True,
                     max_length=max_target_length,
                     truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [18]:
tokenized_dataset = dataset.map(preprocess_data,
                                batched=True)

Map:   0%|          | 0/601 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [19]:
tokenized_dataset.keys()

dict_keys(['train', 'validation', 'test'])

In [20]:
tokenized_dataset = tokenized_dataset.remove_columns(['inputs', 'targets', 'template_lang', 'template_id'])

In [21]:
data_collator = DataCollatorForSeq2Seq(tokenizer,model=model)

# Fine tuning

In [22]:
# parameters
BATCH_SIZE = 8
LR = 5e-5
EPOCHS = 20
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [23]:
# Dataloaders
dataloader_train = torch.utils.data.DataLoader(dataset=tokenized_dataset['train'], collate_fn=data_collator, batch_size=BATCH_SIZE)
dataloader_val = torch.utils.data.DataLoader(dataset=tokenized_dataset['validation'], collate_fn=data_collator, batch_size=BATCH_SIZE)

In [24]:
# training loop
model.to(DEVICE)
optimizer = AdamW(model.parameters(), lr=LR)
lr_scheduler = ReduceLROnPlateau(optimizer, 'min', patience = 5)
writer = SummaryWriter()
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

for epoch in range(EPOCHS):

    print("Train")
    total_loss = 0
    for train_batch in tqdm(dataloader_train):

        model.train()
        train_batch_inputs = {k: v.to(DEVICE) for k, v in train_batch.items()}


        optimizer.zero_grad()

        logits = model(**train_batch_inputs).logits
        loss = criterion(logits.view(-1, logits.shape[-1]), train_batch_inputs["labels"].view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        writer.add_scalar('Train Loss', loss.item(), epoch)

    average_loss = total_loss / len(dataloader_train)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Train Loss: {average_loss}')

    print("Val")
    total_val_loss = 0
    for val_batch in tqdm(dataloader_val):

        model.eval()
        val_batch_inputs = {k: v.to(DEVICE) for k, v in val_batch.items()}

        val_logits = model(**val_batch_inputs).logits
        val_loss = criterion(val_logits.view(-1, val_logits.shape[-1]), val_batch_inputs["labels"].view(-1))


        total_val_loss += val_loss.item()

        lr_scheduler.step(val_loss)
        writer.add_scalar('Validation Loss', loss.item(), epoch)

    average_val_loss = total_val_loss / len(dataloader_val)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Validation Loss: {average_val_loss}')

writer.close()

Train


  0%|          | 0/76 [00:00<?, ?it/s]

Epoch 1/20, Train Loss: 15.17785248003508
Val


  0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1/20, Validation Loss: 3.782116756439209
Train


...

Epoch 19/20, Validation Loss: 3.7808141899108887
Train


  0%|          | 0/76 [00:00<?, ?it/s]

Epoch 20/20, Train Loss: 10.661496965508713
Val


  0%|          | 0/25 [00:00<?, ?it/s]

Epoch 20/20, Validation Loss: 3.780732660293579


In [26]:
model.save_pretrained('/content/drive/MyDrive/gujju-gpt/Models/mT5-gujarati-summarization')

In [29]:
datasets_train_validation['test'][7]

{'inputs': 'નીચેના વાક્યને અલગ રીતે ફરીથી લખો: "કોંગ્રેસ અધ્યક્ષ રાહુલ ગાંધીએ પીએમ મોદીને કેરળમાં આવેલા પૂરને રાષ્ટ્રીય આપત્તિ જાહેર કરવાની માંગ કરી છે."',
 'targets': '"કેરળમાં આવેલા પૂરને રાષ્ટ્રીય આપત્તિ જાહેર કરવાની માંગ કોંગ્રેસ અધ્યક્ષ રાહુલ ગાંધીએ પીએમ મોદીને કરી છે."',
 'template_lang': "['guj']",
 'template_id': 2}

In [31]:
reference_summary = """નીચેના વાક્યને અલગ રીતે ફરીથી લખો: "કોંગ્રેસ અધ્યક્ષ રાહુલ ગાંધીએ પીએમ મોદીને કેરળમાં આવેલા પૂરને રાષ્ટ્રીય આપત્તિ જાહેર કરવાની માંગ કરી છે."""
input = """કેરળમાં આવેલા પૂરને રાષ્ટ્રીય આપત્તિ જાહેર કરવાની માંગ કોંગ્રેસ અધ્યક્ષ રાહુલ ગાંધીએ પીએમ મોદીને કરી છે."""
inputs = tokenizer(input, text_target=output, return_tensors="pt")
hypothesis_summary = tokenizer.batch_decode(model.to('cpu').generate(inputs.input_ids,max_length=50),skip_special_tokens=True)[0]
hypothesis_summary

'<extra_id_0> કરવા માંગ કરી છે.'

In [32]:
datasets_train_validation['test'][10]

{'inputs': 'નીચેના વાક્યને બીજા સ્વરૂપમાં ફરીથી લખો: "22 વર્ષની ઉંમરે, પંતે પહેલાંથી જ ભારતીય ક્રિકેટ ટીમ માટે 13 ટેસ્ટ, 15 ઓડિસ અને 28 ટી 20 આઈ રમ્યા છે."',
 'targets': '"પંતે 22 વર્ષની ઉંમરે પહેલાંથી જ ભારતીય ક્રિકેટ ટીમ માટે 13 ટેસ્ટ, 15 ઓડિસ અને 28 ટી 20 આઈ રમ્યા છે."',
 'template_lang': "['guj']",
 'template_id': 3}

In [33]:
reference_summary = """નીચેના વાક્યને બીજા સ્વરૂપમાં ફરીથી લખો: "22 વર્ષની ઉંમરે, પંતે પહેલાંથી જ ભારતીય ક્રિકેટ ટીમ માટે 13 ટેસ્ટ, 15 ઓડિસ અને 28 ટી 20 આઈ રમ્યા છે."""
input = """પંતે 22 વર્ષની ઉંમરે પહેલાંથી જ ભારતીય ક્રિકેટ ટીમ માટે 13 ટેસ્ટ, 15 ઓડિસ અને 28 ટી 20 આઈ રમ્યા છે."""
inputs = tokenizer(input, text_target=output, return_tensors="pt")
hypothesis_summary = tokenizer.batch_decode(model.to('cpu').generate(inputs.input_ids,max_length=50),skip_special_tokens=True)[0]
hypothesis_summary

'<extra_id_0> છે.'